# Welcome to Antidote

## Introduction

In this tutorial, you will be introduced to Antidote Databases. You can test and see how an eventually consistent database works. We will guide you through the Antidote CRDT data types, transactions, and how Antidote resolves inconsistencies. For more information, see http://syncfree.github.io/antidote/.

> Note that this Jupyter notebook is based on Antidote Java API, but it is tailored for tutorial purpose. As a result, this tutorial might not reflect the full capabilities of Antidote. To find out how to use the Java API, please refer to the __[official Antidote Java API](https://www.javadoc.io/doc/eu.antidotedb/antidote-java-client/0.1.0)__. The appendix at the end of this notebook contains all available functions supported.

First of all, you need to make sure that Antidote service is up and running.

In [ ]:
init()

## Antidote data types

Antidote supports several Conflict-free replicated data types (CRDT). Each type handles inconsistencies differently. In this section, we show some examples of how you can use these data types. The data types supported by Antidote include:
- __Number types__
    - Counter
    - Fat counter
    - Integer
- __Registers__
    - Last-writer wins register
    - Multi-value register
- __Sets__
    - Add-wins set (or Observed-remove set)
    - Remove-win set
- __Maps__
    - Grow-only map
    - Add-wins map
    - Remove-resets map

In Antidote, an object of any data types consists of 2 parts: a key and a value. You need a key in order to update/remove/reset/read your data object in an Antidote bucket. A bucket is an internal data structure used by Antidote where objects are stored. In this tutorial, we have 1 bucket which uses a unique session id as its key. An object key should be unique within the bucket, regardless of data types.

### Number Types

The simplest number data type is Counter. First, we need to create a key for our counter object. We use `"c1"` as a unique key for accessing this Counter object in Antidote database. `getCounterKey("c1")` returns a Counter Key object with key `"c1"`. We assign this Counter Key object to `counter_key` for an easy reference to our `"c1"` key later on.

In [ ]:
counter_key = getCounterKey("c1")

Counter is simply an integer object with increment and decrement operations. The initial value of a counter is 0. Let's try incrementing our `c1` counter by 10, and decrementing it by 1.

In [ ]:
applyUpdate(incrementCounter(counter_key, 10))
applyUpdate(incrementCounter(counter_key, -1))

Reading a counter object returns the aggregated value from all received operations. In our example, the `c1` counter should now be equal to 9.

In [ ]:
read(counter_key)

Fat counter and Integer is similar to Counter, except that, in Fat counter, you can reset the counter to its inital state (value 0), while in Integer, you can assign a value to it.

### Registers

Register can store a single-valued object. We use a register object to store a String in the following example.

In [ ]:
register_key = getLWRegisterKey("r1")

In [ ]:
applyUpdate(assignLWRegister(register_key, "Hello"))

In [ ]:
read(register_key)

### Sets

Set can store a collection of values of the same type. This notebook supports only a Set of Strings.

In [ ]:
set_key = getSetKey("s1")

In [ ]:
applyUpdate(addToSet(set_key, "A","B","C","D"))

In [ ]:
read(set_key)

In [ ]:
applyUpdate(removeFromSet(set_key, "C","D"))

In [ ]:
read(set_key)

### Maps

Map can store a collection of values of different Antidote data types. In the following example, we create a map which stores 2 element objects: an integer, and a fat counter object.

In [ ]:
map_key = getAWMapKey("m1")

In [ ]:
integer_key = getIntegerKey("i1")

In [ ]:
fatcounter_key = getFatCounterKey("f1")

To update a map, we actually update element objects of the map.

In [ ]:
applyUpdate(updateAWMap(map_key, assignInteger(integer_key, 1)
                               , incrementFatCounter(fatcounter_key, 2)))

To read from a map, we also need to specify the key of the element object.

In [ ]:
readFromMap(map_key, integer_key)

In [ ]:
readFromMap(map_key, fatcounter_key)

In [ ]:
applyUpdate(removeFromAWMap(map_key, integer_key))

In [ ]:
readFromMap(map_key, integer_key)

In [ ]:
readFromMap(map_key, fatcounter_key)

We can also store maps in a map 

In [ ]:
child_map_key = getAWMapKey("m1.1")

In [ ]:
child_integer_key = getIntegerKey("i1.1")

In [ ]:
child_counter_key = getCounterKey("c1.3")

In [ ]:
applyUpdate(updateAWMap(map_key, updateAWMap(child_map_key, 
                            assignInteger(child_integer_key, 43), incrementCounter(child_counter_key, 3))))

Reading a map in a map will return a mapResult object

In [ ]:
mapResult = readFromMap(map_key, child_map_key)

mapResults can be read seperately. And it is possible to use it recursively to read map results. 

In [ ]:
readFromMapResult(mapResult, child_counter_key)

## Transactions

So far, we use `applyUpdate()` to update the Antidote objects, but you can also group several updates to create an atomic transaction. Let's see an example below. We create a transaction that will assign a value to `x` and incremrent `y`.

In [ ]:
x = getIntegerKey("x")
y = getCounterKey("y")

First, we have to get a transaction object from `startTransaction()`.

In [ ]:
tx = startTransaction()

Any updates to be included in the transaction must be called with `applyUpdateWithTransaction()`.

In [ ]:
applyUpdateWithTransaction(tx, incrementInteger(x, 10))

In [ ]:
applyUpdateWithTransaction(tx, incrementCounter(y, 1))

With `applyUpdateWithTransaction()`, the update is applied to the local Antidote node only. `readInTransaction()` returns the current value if all update operations in the specified transaction are applied.

In [ ]:
readInTransaction(tx,x)

When you still have not committed your transaction, the updates will not be applied globally. That's why `read(x)` returns null, instead of 10.

In [ ]:
read(x)

Let's see what happens after we commit the transaction.

In [ ]:
commitTransaction(tx)

Now the updates in the transaction are applied globally.

In [ ]:
read(x)

In [ ]:
read(y)

> In this tutorial, `applyUpdate()` is an update with `NoTransaction`, while `applyUpdateWithTransaction()` is an update with `InteractiveTransaction` type. You can find out more about the other Antidote transaction types from the __[official Antidote Java API](https://www.javadoc.io/doc/eu.antidotedb/antidote-java-client/0.1.0)__.

## Data replication

In Antidote, replicated databases are eventually consistent. Each data type resolves the consistencies differently. In this section, we will demonstrate this by using 2 replicated Antidote nodes: `Antidote1` and `Antidote2`. By default, this notebook connects to `Antidote1`.

<img src="images/Antidote-Notebook-diagram.png">

We first create an Integer Key `"x"` on `Antidote1`.

In [ ]:
switchAntidote(1)
x = getIntegerKey("x")
applyUpdate(assignInteger(x, 2))
read(x)

Both `Antidote1` and `Antidote2` are currently connected. Any updates to either one of them will be replicated to the other one. Now we switch to Antidote2 to see if `x` is 2 as well.

In [ ]:
switchAntidote(2)

In [ ]:
read(x)

## Inconsistency resolution

In this section, we will see how Antidote resolves conflicts when updates are done on disconnected replicas.

### Last-writer-wins

If there are multiple updates while Antidote nodes are disconnected, the final value of the object will be the value updated at the last timestamp. The Last-writer-wins register uses this mechanism.

We first create a Last-writer-wins register key when both Antidote nodes are connected.

In [ ]:
connectAntidotes()
lww_key = getLWRegisterKey("lww1")
applyUpdate(assignLWRegister(lww_key, "A"))
read(lww_key)

We then disconnect the nodes to see how Antidote handles inconsistencies.

In [ ]:
disconnectAntidotes()

`disconnectAntidotes()` and `connectAntidotes()` simulates a disconnection and connection between the 2 Antidote nodes. <img src="images/Antidote-Notebook-diagram-disconnected.png">

In [ ]:
switchAntidote(1)
applyUpdate(assignLWRegister(lww_key, "B"))
read(lww_key)

In [ ]:
switchAntidote(2)
applyUpdate(assignLWRegister(lww_key, "C"))
read(lww_key)

While `Antidote1` and `Antidote2` are disconnected, key `lww1` was updated differently. This causes an inconsistency to key `lww1`.

In [ ]:
connectAntidotes()

After re-connecting, Antidote resolves the inconsistencies using Last-writer-wins mechanism. The expected value stored in key `lww1` is `"C"` since it is the last update performed on key `lww1`.

In [ ]:
switchAntidote(1)
read(lww_key)

### Multi-value

If there are multiple updates while Antidote nodes are disconnected, all of the last updates on different Antidote nodes will be included the final values. The multi-value uses this mechanism. The final list is ordered in alphabetical order. In its initial state, reading the register returns the empty list.

In [ ]:
connectAntidotes()
mv_key = getMVRegisterKey("mv1")
applyUpdate(assignMVRegister(mv_key, "A"))
read(mv_key)

In [ ]:
disconnectAntidotes()

In [ ]:
switchAntidote(1)
applyUpdate(assignMVRegister(mv_key, "B"))

In [ ]:
switchAntidote(2)
applyUpdate(assignMVRegister(mv_key, "C"))
applyUpdate(assignMVRegister(mv_key, "D"))

In [ ]:
connectAntidotes()

The expected values stored on key mv1 is `["B","D"]` since both were the last update on different Antidote nodes during disconnection.

In [ ]:
switchAntidote(1)
read(mv_key)

### Add-wins

Add-operations win over concurrent remove-operations. In this context, concurrent operations refer to operations committed to any Antidote nodes while they are not connected. A remove-operation will only “overwrite” the add-operations that happened before the Antidote nodes are disconnected.

In [ ]:
connectAntidotes()
aw_key = getSetKey("aw1")
applyUpdate(addToSet(aw_key, "A"))
read(aw_key)

In [ ]:
disconnectAntidotes()

In [ ]:
switchAntidote(1)
applyUpdate(addToSet(aw_key, "B"))

In [ ]:
switchAntidote(2)
applyUpdate(removeFromSet(aw_key, "A"))
applyUpdate(removeFromSet(aw_key, "B"))

In [ ]:
connectAntidotes()

The expected values stored on key aw_key is `["B"]`. 
- `"A"` was successfully removed because there was no concurrent adding attempt on `"A"`. 
- `"B"` was included in the final value because there were concurrent adding and removing attempts on `"B"`, and add-operations win over remove-operations.

In [ ]:
switchAntidote(1)
read(aw_key)

### Remove-wins

Remove-operations win over concurrent add-operations. In this context, concurrent operations refer to operations committed to any Antidote nodes while they are not connected. An add-operation overwrites only the remove-operations that happened before the Antidote nodes are disconnected.

In [ ]:
connectAntidotes()
rw_key = getRWSetKey("rw1")
applyUpdate(addToSet(rw_key, "A"))
read(rw_key)

In [ ]:
disconnectAntidotes()

In [ ]:
switchAntidote(1)
applyUpdate(removeFromSet(rw_key, "B"))

In [ ]:
switchAntidote(2)
applyUpdate(addToSet(rw_key, "B"))

In [ ]:
connectAntidotes()

The expected value stored on key aw_key is only `["A"]`.
- `"A"` was included in the final value because there was no update attempt on `"A"` during disconnection.
- `"B"` was successfully removed because there were concurrent adding and removing attempts on `"B"`, and remove-operations win over add-operations.

In [ ]:
switchAntidote(1)
read(rw_key)

### Appendix

Following are all the method that are available for use in this notebook. 


- Initializes the Antidote client and connects the client to Antidote node 1. Creates a bucket with a unique key that will be used by this client. 

```init()```

- Switch the session to the specified antidote node. Param node can either be 1 or 2. 
    - param antidoteNode

```switchAntidote(1)```

- Connect the antidote DB replicas. 

```connectAntidotes()```

- Disrupt the connection between the antidote DB replicas. 

```disconnectAntidotes()```

- Creates a Antidote Interactive Transaction
    - Return AntidoteTransaction
    
```startTransaction()```

- Adds an update operation to an interactive transaction. 
    - Params: AntidoteTransaction tx, UpdateOp updateOp

```applyUpdateWithTransaction(tx, updateOp)```

- Commits an transaction. This will execute all the update operations added to the transaction.
    - Param AntidoteTransaction tx
    
```commitTransaction(tx)```

- Reads a key stored in the antidote database. 
    - Param Key<T>
    - Return object read from the key. 
    
```read(key)```


#### LW Register

- Create a Last Wins Register key object with the passed string as the key ID.
    - Param: String keyId
    - Return RegisterKey<String>
    
```getLWRegisterKey("keyId")```

- Updates the value of the register with the passed registerKey.
    - Param RegisterKey<String>, String value
    - Return UpdateOp
    
```assignLWRegister(registerKey, "value")```

#### MV Register

- Create a Multi Value Register key object with the passed string as the key ID. 
    - Param String keyId
    - Return MVRegisterKey<String>
    
```getMVRegisterKey("keyId")```

- Updates the value of the register with the passed registerKey. 
    - Param MVRegisterKey<String>, String value
    - Return UpdateOp
    
```assignMVRegister(MVregisterKey, "value")```

- Reset the MV register to initial state (empty list) 
    - Param MVRegisterKey<String>
    - Return UpdateOp
    
```resetMVRegister(MVregisterKey)```

#### SET

- Create a Add Wins Set key object with the given keyId. 
    - Param String keyId
    - Return SetKey<String>
    
```getSetKey("keyId")```

- Add the values to the set with the given key.  
    - Param SetKey<String>, String…
    - Return UpdateOp
    
```addToSet(setKey, "value1", "value2"...)```

- Remove the specified values from the set with given key
    - Param SetKey<String>, String…
    - Return UpdateOp
    
```removeFromSet(setKey, "value1", "value2"...)```

- Resets the set with the given key. 
    - Param SetKey<String>
    - Return UpdateOp
    
```resetSet(setKey)```

#### RW SET

- Create a Remove Wins Set key object with the given keyId.
    - Param String keyId
    - Return SetKey<String>
    
```getRWSetKey("keyId")```

- Add the values to the Remove Wins Set with the given key.   
    - Param SetKey<String>, String…
    - Return UpdateOp
    
```addToRWSet(setKey, "value1", "value2")```

- Remove the specified values from the Remove Wins set with given key.
    - Param SetKey<String>, String…
    - Return UpdateOp
    
```removeFromRWSet(setKey, "value1", "value2")```

- Resets the Remove Wins set with the given key.  
    - Param SetKey<String>
    - Return UpdateOp
    
```resetRWSet(setKey)```

#### Integer

- Create an IntegerKey object with the given key ID 
    - Param String keyId
    - Return IntegerKey
    
```getIntegerKey("keyId")```

- Assign the value of the integer. 
    - Param IntegerKey, int value
    - Return UpdateOp
    
```assignInteger(integerKey, 1)```

- Increment the integer with the given amount. 
    - Param IntegerKey, int increment
    - Return UpdateOp
    
```incrementInteger(integerKey, increment)```

#### Counter

- Create an CounterKey object with the given key ID 
    - Param String keyId
    - Return CounterKey
    
```getCounterKey("keyId")```

- Increment the Counter with the given amount. 
    - Param CounterKey, int increment
    - Return UpdateOp
    
```incrementCounter(counterKey, increment)```

#### Fat Counter

- Create an FatCounterKey object with the given key ID 
    - Param String keyId
    - Return FatCounterKey
    
```getFatCounterKey("keyId")```

- Increment the FatCounter with the given amount. 
    - Param FatCounterKey, int increment
    - Return UpdateOp
    
```incrementFatCounter(counterKey, increment)```

- Resets the FatCounter with the given key.  
    - Param FatCounterKey
    - Return UpdateOp
    
```resetFatCounter(fatCounterKey)```

#### AW Map

- Create an Add Wins Map Key object with the given key ID 
    - Param String keyId
    - Return MapKey
    
```getAWMapKey("keyId")```

- Add elements to the Add Wins Map. Elements are added as update operations on their keys.  
    - Param MapKey, UpdateOp...
    - Return UpdateOp
    
```updateAWMap(mapKey, updateOp1, updateOp2)```

- Remove the specified elements from the Add Wins Map with given key. Elements are passed as their key objects.
    - Param MapKey, Key…
    - Return UpdateOp
    
```removeFromAWMap(mapKey, key1, key2)```

#### RR Map

- Create an Remove Resets Map Key object with the given key ID 
    - Param String keyId
    - Return MapKey
    
```getRRMapKey("keyId")```

- Add elements to the Remove Resets Map. Elements are added as update operations on their keys.  
    - Param MapKey, UpdateOp...
    - Return UpdateOp
    
```updateRRMap(mapKey, updateOp1, updateOp2)```

- Remove the specified elements from the Remove Resets Map with given key. Elements are passed as their key objects.
    - Param MapKey, Key…
    - Return UpdateOp
    
```removeFromRRMap(mapKey, key1, key2)```

- Resets the Remove Resets Map with the given key.  
    - Param RRMapKey
    - Return UpdateOp
    
```resetRRMap(rrMapKey)```

#### G Map

- Create an G Map Key object with the given key ID 
    - Param String keyId
    - Return MapKey
    
```getGMapKey("keyId")```

- Add elements to the G Map. Elements are added as update operations on their keys.  
    - Param MapKey, UpdateOp...
    - Return UpdateOp
    
```updateGMap(mapKey, updateOp1, updateOp2)```